# Project Group 2

## Import libraries

In [7]:
import pandas as pd
import numpy as np
from pathlib import Path

import requests
from bs4 import BeautifulSoup

from selenium import webdriver
from selenium.webdriver.common.by import By

from time import sleep

import matplotlib.pyplot as plt
import seaborn as sns

In [183]:
driver = webdriver.Chrome()  # Instances WebDriver for Chrome

driver.get("https://airbnb.com")  # Opens Airbnb website

sleep(5)  # Waits for 5 seconds

# Clicks on the Accept button for cookies if it appears
try:
    driver.find_element(
        By.XPATH,
        value="/html/body/div[5]/div/div/div[1]/div/div[6]/section/div[2]/div[2]/button",
    ).click()
except:
    pass

In [184]:
city = "Valencia"  # City to search for

# Finds the search input and types the city

driver.find_element(
    By.ID,
    value="bigsearch-query-location-input",
).send_keys(city)

sleep(1)  # Waits for 1 second

# Clicks on the search button

driver.find_element(By.CLASS_NAME, value="bhtghtc").click()

In [185]:
# Store the ID of the original window
original_window = driver.current_window_handle

# Initializes the lists to store the data

name = list()
type = list()
price_per_night = list()
host = list()
traveller_recommendation = list()
rating = list()
reviews = list()

In [186]:
driver.switch_to.window(driver.window_handles[0])

driver.refresh()

In [187]:
next_page = driver.find_element(
    By.XPATH,
    value="""
    /html/body/div[5]/div/div/div[1]/div/div[3]/div[2]
    /main/div[2]/div/div[3]/div/div/div/nav/div/a[5]
    """,
)

while next_page != None:

    # Iterates over the listings and opens each one in a new tab. Then, it closes the tab and goes back to the original window

    listings = driver.find_elements(
        By.CLASS_NAME, value="c4mnd7m"
    )  # Finds all the listings

    for listing in listings:
        listing.click()  # Clicks on the listing
        sleep(2)
        driver.switch_to.window(driver.window_handles[1])  # Switches to the new tab

        sleep(5)

        # Sometimes, a pop-up appears. This code closes it if it does
        try:
            driver.find_element(
                By.XPATH,
                value="/html/body/div[9]/div/div/section/div/div/div[2]/div/div[1]/button",
            ).click()
        except:
            pass

        """
        Here is it supposed to scrape the data from the listing.
        """

        sleep(2)

        # Getting the data

        # Name of the listing
        try:
            name.append(
                driver.find_element(
                    By.XPATH,
                    value="""
                    /html/body/div[5]/div/div/div[1]/div/div[2]/div/div/div/div[1]/main
                    /div/div[1]/div[1]/div[1]/div/div/div/div/div/section/div/div[1]/span/h1
                    """,
                ).text
            )
        except:
            name.append(np.nan)

        # -------------------------
        # Type of accommodation

        try:
            type.append(
                driver.find_element(
                    By.XPATH,
                    value="""
                    /html/body/div[5]/div/div/div[1]/div/div[2]/div/div/div/div[1]/main
                    /div/div[1]/div[3]/div/div[1]/div/div[1]/div/div/div/section/div[1]/h2
                    """,
                ).text
            )
        except:
            type.append(np.nan)

        # -------------------------
        # Price per night

        try:
            try:
                price_per_night.append(
                    driver.find_elements(By.CLASS_NAME, value="_tyxjp1")[1].text.split(
                        " "
                    )[0]
                )
            except:
                price_per_night.append(
                    driver.find_element(
                        By.XPATH,
                        value="""
                        /html/body/div[5]/div/div/div[1]/div/div[2]/div/div/div/div[1]/main
                        /div/div[1]/div[3]/div/div[2]/div/div/div[1]/div/div/div/div/div/div/div/div[1]/div[1]/div/div/span/div/span[2]
                        """,
                    ).text.split(" ")[0]
                )
        except:
            price_per_night.append(np.nan)

        # -------------------------
        # Host name

        try:
            try:
                host.append(
                    driver.find_element(
                        By.XPATH,
                        value="""
                        /html/body/div[5]/div/div/div[1]/div/div[2]/div/div/div/div[1]/main
                        /div/div[1]/div[3]/div/div[1]/div/div[2]/div[2]/div/div/div/div[2]/div[1]
                        """,
                    ).text
                )

            # When the accommodation is marked as "Traveller recommended", the host name is in a different place
            except:
                host.append(
                    driver.find_element(
                        By.XPATH,
                        value="""
                        /html/body/div[5]/div/div/div[1]/div/div[2]/div/div/div/div[1]/main
                        /div/div[1]/div[3]/div/div[1]/div/div[3]/div/div/div/div/div[2]/div[1]
                        """,
                    ).text
                )
        except:
            host.append(np.nan)

            # -------------------------
            # Rating

        try:
            trav_recom = driver.find_element(
                By.XPATH,
                value="""
                /html/body/div[5]/div/div/div[1]/div/div[2]/div/div/div/div[1]/main
                /div/div[1]/div[3]/div/div[1]/div/div[2]/div/div/div/a/div/div[2]/div/div[2]
                """,
            )
        except:
            trav_recom = None

        if trav_recom == None:
            traveller_recommendation.append(0)
            try:
                rating.append(
                    driver.find_element(
                        By.XPATH,
                        value="""
                    /html/body/div[5]/div/div/div[1]/div/div[2]/div/div/div/div[1]/main
                    /div/div[1]/div[3]/div/div[1]/div/div[1]/div/div/div/section/div[3]/div[2]
                    """,
                    ).text
                )
            except:
                rating.append(np.nan)

        else:
            traveller_recommendation.append(1)
            try:
                rating.append(
                    driver.find_element(
                        By.XPATH,
                        value="""
                        /html/body/div[5]/div/div/div[1]/div/div[2]/div/div/div/div[1]/main
                        /div/div[1]/div[3]/div/div[1]/div/div[2]/div/div/div/a/div/div[6]/div[1]
                        """,
                    ).text
                )
            except:
                rating.append(np.nan)

            # -------------------------
            # Reviews

        if trav_recom == None:
            try:
                reviews.append(
                    driver.find_element(
                        By.XPATH,
                        value="""
                    /html/body/div[5]/div/div/div[1]/div/div[2]/div/div/div/div[1]/main
                    /div/div[1]/div[3]/div/div[1]/div/div[1]/div/div/div/section/div[3]/a
                    """,
                    ).text.split(" ")[0]
                )

            except:
                reviews.append(np.nan)
        else:

            try:
                reviews.append(
                    driver.find_element(
                        By.XPATH,
                        value="""
                        /html/body/div[5]/div/div/div[1]/div/div[2]/div/div/div/div[1]/main
                        /div/div[1]/div[3]/div/div[1]/div/div[2]/div/div/div/a/div/div[10]/div[1]
                        """,
                    ).text
                )
            except:
                reviews.append(np.nan)

        sleep(2)

        driver.close()
        sleep(1)
        driver.switch_to.window(original_window)

    next_page.click()

    sleep(5)

    try:
        next_page = driver.find_element(
            By.XPATH,
            value="""
            /html/body/div[5]/div/div/div[1]/div/div[3]/div[2]
            /main/div[2]/div/div[3]/div/div/div/nav/div/a[6]
            """,
        )
    except:
        next_page = None

    sleep(1)

KeyboardInterrupt: 

In [188]:
name, type, price_per_night, host, rating, reviews

(['Bajo a estrenar en pleno centro',
  'Piso increíble en pleno Carmen',
  'Alojamiento entero',
  'Amplio, cómodo y ubicadísimo!',
  'Villa con encanto cerca de Valencia capital',
  'Un lugar tranquilo y familiar',
  'Gran loft',
  'Super Apartamento de 3 habitaciones, 2 dormitorios, wifi de aire acondicionado',
  'Luminoso, cómodo, céntrico 2',
  'Fantástico apartamento entero',
  'Sueños de Valencia',
  'PISO PLAZA DE LA REINA Free Cov. OZONO',
  'apartamento con terraza',
  'Apartamento en Patraix',
  'El Mercat 02',
  'Av Puerto amplio bien comunicado',
  'Piso en Ruzafa, Valencia,2 habitaciones, 2 baños',
  'El rincón de los sueños Valencia',
  'Centro de la ciudad Valencia 1',
  'Piso luminoso a 5 minutos de la Playa.'],
 ['Alojamiento entero: piso en València, España',
  'Alojamiento entero: piso en València, España',
  'Alojamiento entero: piso en València, España',
  'Alojamiento entero: piso en València, España',
  'Alojamiento entero: vivienda en Valencia, España',
  'Aloja

In [191]:
airbnb_data = pd.DataFrame(
    {
        "Name": name,
        "Type": type,
        "Price per night": price_per_night,
        "Host": host,
        "traveller_recommendation": traveller_recommendation,  # 1 if it is recommended, 0 if it is not
        "Rating": rating,
        "Reviews": reviews,
    }
)

In [192]:
airbnb_data

,Name,Type,Price per night,Host,traveller_recommendation,Rating,Reviews
0,Bajo a estrenar en pleno centro,"Alojamiento entero: piso en València, España",214,Anfitrión: Federico,0,Nuevo,1
1,Piso increíble en pleno Carmen,"Alojamiento entero: piso en València, España",400,Anfitrión: Pilar,0,Nuevo,NaN
2,Alojamiento entero,"Alojamiento entero: piso en València, España",96,Anfitrión: Martín Daniel,0,Nuevo,1
3,"Amplio, cómodo y ubicadísimo!","Alojamiento entero: piso en València, España",88,Anfitrión: María Del Rocío,0,Nuevo,1
4,Villa con encanto cerca de Valencia capital,"Alojamiento entero: vivienda en Valencia, España",260,Anfitrión: Kely,0,Nuevo,NaN
5,Un lugar tranquilo y familiar,"Alojamiento entero: piso en València, España",78,Anfitrión: Jeison,0,"5,0",3
6,Gran loft,"Alojamiento entero: piso en València, España",90,Anfitrión: Josue,0,Nuevo,1
7,"Super Apartamento de 3 habitaciones, 2 dormito...","Alojamiento entero: piso en València, España",120,Anfitrión: Alfonso,0,Nuevo,NaN
8,"Luminoso, cómodo, céntrico 2","Alojamiento entero: loft en València, España",117,Anfitrión: Mr Flats,0,"4,54",120
9,Fantástico apartamento entero,"Alojamiento entero: piso en València, España",150,Anfitrión: Mercedes,0,Nuevo,NaN


In [59]:
listings = driver.find_elements(
    By.CLASS_NAME, value="c4mnd7m"
)  # Finds all the listings

listings[6].click()

In [178]:
driver.switch_to.window(driver.window_handles[1])

driver.refresh()

sleep(5)

try:
    trav_recom = driver.find_element(
        By.XPATH,
        value="""
        /html/body/div[5]/div/div/div[1]/div/div[2]/div/div/div/div[1]/main
        /div/div[1]/div[3]/div/div[1]/div/div[2]/div/div/div/a/div/div[2]/div/div[2]
        """,
    )
except:
    trav_recom = None

if trav_recom == None:
    try:

        print(
            driver.find_element(
                By.XPATH,
                value="""
        /html/body/div[5]/div/div/div[1]/div/div[2]/div/div/div/div[1]/main
        /div/div[1]/div[3]/div/div[1]/div/div[1]/div/div/div/section/div[3]/div[2]
        """,
            ).text
        )
    except:
        print(np.nan)
else:
    try:
        print(trav_recom.text)
        print(
            driver.find_element(
                By.XPATH,
                value="""
        /html/body/div[5]/div/div/div[1]/div/div[2]/div/div/div/div[1]/main
        /div/div[1]/div[3]/div/div[1]/div/div[2]/div/div/div/a/div/div[6]/div[1]
        """,
            ).text
        )
    except:
        print(np.nan)

Recomendación
del viajero
4,97


In [ ]:
driver.switch_to.window(driver.window_handles[1])

driver.refresh()

sleep(5)

try:
    trav_recom = driver.find_element(
        By.XPATH,
        value="""
        /html/body/div[5]/div/div/div[1]/div/div[2]/div/div/div/div[1]/main
        /div/div[1]/div[3]/div/div[1]/div/div[2]/div/div/div/a/div/div[2]/div/div[2]
        """,
    )
except:
    trav_recom = None

if trav_recom == None:
    try:

        print(
            driver.find_element(
                By.XPATH,
                value="""
        /html/body/div[5]/div/div/div[1]/div/div[2]/div/div/div/div[1]/main
        /div/div[1]/div[3]/div/div[1]/div/div[1]/div/div/div/section/div[3]/div[2]
        """,
            ).text
        )
    except:
        print(np.nan)
else:
    try:
        print(trav_recom.text)
        print(
            driver.find_element(
                By.XPATH,
                value="""
        /html/body/div[5]/div/div/div[1]/div/div[2]/div/div/div/div[1]/main
        /div/div[1]/div[3]/div/div[1]/div/div[2]/div/div/div/a/div/div[6]/div[1]
        """,
            ).text
        )
    except:
        print(np.nan)

Recomendación
del viajero
4,97


In [ ]:
driver.switch_to.window(driver.window_handles[1])

driver.refresh()

sleep(5)

try:
    trav_recom = driver.find_element(
        By.XPATH,
        value="""
        /html/body/div[5]/div/div/div[1]/div/div[2]/div/div/div/div[1]/main
        /div/div[1]/div[3]/div/div[1]/div/div[2]/div/div/div/a/div/div[2]/div/div[2]
        """,
    )
except:
    trav_recom = None

if trav_recom == None:
    try:

        print(
            driver.find_element(
                By.XPATH,
                value="""
        /html/body/div[5]/div/div/div[1]/div/div[2]/div/div/div/div[1]/main
        /div/div[1]/div[3]/div/div[1]/div/div[1]/div/div/div/section/div[3]/div[2]
        """,
            ).text
        )
    except:
        print(np.nan)
else:
    try:
        print(trav_recom.text)
        print(
            driver.find_element(
                By.XPATH,
                value="""
        /html/body/div[5]/div/div/div[1]/div/div[2]/div/div/div/div[1]/main
        /div/div[1]/div[3]/div/div[1]/div/div[2]/div/div/div/a/div/div[6]/div[1]
        """,
            ).text
        )
    except:
        print(np.nan)

Recomendación
del viajero
4,97


In [ ]:
driver.switch_to.window(driver.window_handles[1])

driver.refresh()

sleep(5)

try:
    trav_recom = driver.find_element(
        By.XPATH,
        value="""
        /html/body/div[5]/div/div/div[1]/div/div[2]/div/div/div/div[1]/main
        /div/div[1]/div[3]/div/div[1]/div/div[2]/div/div/div/a/div/div[2]/div/div[2]
        """,
    )
except:
    trav_recom = None

if trav_recom == None:
    try:

        print(
            driver.find_element(
                By.XPATH,
                value="""
        /html/body/div[5]/div/div/div[1]/div/div[2]/div/div/div/div[1]/main
        /div/div[1]/div[3]/div/div[1]/div/div[1]/div/div/div/section/div[3]/div[2]
        """,
            ).text
        )
    except:
        print(np.nan)
else:
    try:
        print(trav_recom.text)
        print(
            driver.find_element(
                By.XPATH,
                value="""
        /html/body/div[5]/div/div/div[1]/div/div[2]/div/div/div/div[1]/main
        /div/div[1]/div[3]/div/div[1]/div/div[2]/div/div/div/a/div/div[6]/div[1]
        """,
            ).text
        )
    except:
        print(np.nan)

Recomendación
del viajero
4,97


In [ ]:
next_page = driver.find_element(
    By.XPATH,
    value="""
    /html/body/div[5]/div/div/div[1]/div/div[3]/div[2]
    /main/div[2]/div/div[3]/div/div/div/nav/div/a[5]
    """,
)

while next_page != None:

    # Iterates over the listings and opens each one in a new tab. Then, it closes the tab and goes back to the original window

    listings = driver.find_elements(
        By.CLASS_NAME, value="c4mnd7m"
    )  # Finds all the listings

    for listing in listings:
        listing.click()  # Clicks on the listing
        sleep(2)
        driver.switch_to.window(driver.window_handles[1])  # Switches to the new tab

        sleep(5)

        # Sometimes, a pop-up appears. This code closes it if it does
        try:
            driver.find_element(
                By.XPATH,
                value="/html/body/div[9]/div/div/section/div/div/div[2]/div/div[1]/button",
            ).click()
        except:
            pass

        """
        Here is it supposed to scrape the data from the listing.
        """

        sleep(2)

        # Getting the data

        # Name of the listing
        try:
            name.append(
                driver.find_element(
                    By.XPATH,
                    value="""
                    /html/body/div[5]/div/div/div[1]/div/div[2]/div/div/div/div[1]/main
                    /div/div[1]/div[1]/div[1]/div/div/div/div/div/section/div/div[1]/span/h1
                    """,
                ).text
            )
        except:
            name.append(np.nan)

        # -------------------------
        # Type of accommodation

        try:
            type.append(
                driver.find_element(
                    By.XPATH,
                    value="""
                    /html/body/div[5]/div/div/div[1]/div/div[2]/div/div/div/div[1]/main
                    /div/div[1]/div[3]/div/div[1]/div/div[1]/div/div/div/section/div[1]/h2
                    """,
                ).text
            )
        except:
            type.append(np.nan)

        # -------------------------
        # Price per night

        try:
            try:
                price_per_night.append(
                    driver.find_elements(By.CLASS_NAME, value="_tyxjp1")[1].text.split(
                        " "
                    )[0]
                )
            except:
                price_per_night.append(
                    driver.find_element(
                        By.XPATH,
                        value="""
                        /html/body/div[5]/div/div/div[1]/div/div[2]/div/div/div/div[1]/main
                        /div/div[1]/div[3]/div/div[2]/div/div/div[1]/div/div/div/div/div/div/div/div[1]/div[1]/div/div/span/div/span[2]
                        """,
                    ).text.split(" ")[0]
                )
        except:
            price_per_night.append(np.nan)

        # -------------------------
        # Host name

        try:
            host.append(
                driver.find_element(
                    By.XPATH,
                    value="""
                    /html/body/div[5]/div/div/div[1]/div/div[2]/div/div/div/div[1]/main
                    /div/div[1]/div[3]/div/div[1]/div/div[2]/div[2]/div/div/div/div[2]/div[1]
                    """,
                ).text.split(" ")[-1]
            )
        except:
            host.append(np.nan)

        # -------------------------
        # Rating

        try:
            rating.append(
                driver.find_element(
                    By.XPATH,
                    value="""
                    /html/body/div[5]/div/div/div[1]/div/div[2]/div/div/div/div[1]/main
                    /div/div[1]/div[3]/div/div[1]/div/div[1]/div/div/div/section/div[3]/div[2]
                    """,
                ).text
            )
        except:
            rating.append(np.nan)

        # -------------------------
        # Reviews

        try:
            reviews.append(
                driver.find_element(
                    By.XPATH,
                    value="""
                    /html/body/div[5]/div/div/div[1]/div/div[2]/div/div/div/div[1]/main
                    /div/div[1]/div[3]/div/div[1]/div/div[1]/div/div/div/section/div[3]/a
                    """,
                ).text.split(" ")[0]
            )
        except:
            reviews.append(np.nan)

        sleep(2)

        driver.close()
        sleep(1)
        driver.switch_to.window(original_window)

    next_page.click()

    sleep(5)

    try:
        next_page = driver.find_element(
            By.XPATH,
            value="""
            /html/body/div[5]/div/div/div[1]/div/div[3]/div[2]
            /main/div[2]/div/div[3]/div/div/div/nav/div/a[6]
            """,
        )
    except:
        next_page = None

    sleep(1)